# vtab_1k

> https://google-research.github.io/task_adaptation/

https://paperswithcode.com/sota/image-classification-on-vtab-1k-1

In [ ]:
#| default_exp data.vtab_1k

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()